In [1]:
import chipwhisperer as cw
import matplotlib.pyplot as plt

n = 5

scope = cw.scope()
target = cw.target(scope)

scope.default_setup()

cw.program_target(scope, cw.programmers.STM32FProgrammer, "./fizzy-CWNANO.hex")

scope.arm()

scope.adc.samples = 10000


See https://chipwhisperer.readthedocs.io/en/latest/api.html#firmware-update


Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 13243 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 13243 bytes


In [2]:
import time
import numpy as np
# scope.adc.samples = 10000
def reset_target(scope): 
    scope.io.nrst = 'low'
    time.sleep(0.01)
    scope.io.nrst = 'high'
    time.sleep(0.01)

def cap_pass_trace(pass_guess):
    ret = ""
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 16)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
#     target.write(pass_guess)
    target.simpleserial_write('s', pass_guess)

    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace(as_int=True)
    return np.array(trace)

In [3]:
%matplotlib tk
trace = np.array(cap_pass_trace(bytearray([])))
plt.plot(trace)
plt.show()

In [4]:
from numpy import genfromtxt
pattern = genfromtxt('pat2.csv', delimiter=',')
pattern = pattern[3:]
# pattern = trace[37:50]
len_arr = len(pattern)

# res = {}

temp_list = []
for i in range(len(trace)-len_arr):
    temp = trace[i:i+len_arr]
    cf = np.corrcoef(pattern,temp)[0][1]
    temp_list.append(cf)
    
# index = np.argmax(np.array(temp_list))
# a = res.get(index,[])
# a.append(j)
# res[index]=a
# print(f"j: {j}\t index: {index} \t cf: {temp_list[index]}")

In [5]:
%matplotlib tk
from matplotlib.patches import Rectangle
fig, ax = plt.subplots(2,sharex=True)
ax[0].plot(trace)
for i,ele in enumerate(np.array(temp_list)):
    if ele > 0.97 :
#         print(i)
        rect = Rectangle((i, 0), len(pattern), 256, facecolor='lightgrey')
        ax[0].add_patch(rect)
ax[1].plot(temp_list)
plt.show()

In [6]:
# %matplotlib inline
# plt.plot(pattern)
# plt.show()

In [7]:
swap = 0
l = []
for i,ele in enumerate(np.array(temp_list)):
    if ele > 0.97 :
        l.append(swap>0)
        swap=0
    elif ele > 0.92:
        swap += 1
l.append(swap>0)

In [8]:
def swapPositions(list, pos1, pos2):
     
    list[pos1], list[pos2] = list[pos2], list[pos1]
    return list

def find_arr(is_swap,n):
    is_swap.reverse()
    arr = [i for i in range(n)]
    k = 0
    for i in range(n-1):
        for j in range(i+1,0,-1):
            if is_swap[k] :
                arr = swapPositions(arr,j,j-1)
            k += 1
    return arr
    

In [9]:
l[1:]

[True, True, False, True, False, False, True, False, True, True]

In [10]:
per = find_arr(l[1:],n)
# find_arr([0,0,1,1,1,1,0,1,1,1],5)

In [11]:
per

[3, 1, 2, 4, 0]

In [12]:
r = target.simpleserial_read('r',n)

In [13]:
print(r)

CWbytearray(b'00 01 02 03 04')


In [14]:
cw.program_target(scope, cw.programmers.STM32FProgrammer, "./fizzy-CWNANO.hex")

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 13243 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 13243 bytes


In [15]:
msg_fin = bytearray([int(r[i]) for i in per])
target.simpleserial_write('a',msg_fin)
print(target.simpleserial_read('r',1))

CWbytearray(b'00')


In [16]:
[int(r[i]) for i in per]

[3, 1, 2, 4, 0]